In [ ]:
#Importing Libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential, load_model, Model
from tensorflow.keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# !unrar x "/content/drive/MyDrive/train_set.rar" "/content/train_set/"
!unzip "/content/drive/MyDrive/ourDataset.zip" -d "/content/"

Archive:  /content/drive/MyDrive/ourDataset.zip
   creating: /content/train_set/
   creating: /content/train_set/aug_happy/
  inflating: /content/train_set/aug_happy/1001.png  
  inflating: /content/train_set/aug_happy/1002.png  
  inflating: /content/train_set/aug_happy/1003.png  
  inflating: /content/train_set/aug_happy/1004.png  
  inflating: /content/train_set/aug_happy/1005.png  
  inflating: /content/train_set/aug_happy/1006.png  
  inflating: /content/train_set/aug_happy/1007.png  
  inflating: /content/train_set/aug_happy/1008.png  
  inflating: /content/train_set/aug_happy/1009.png  
  inflating: /content/train_set/aug_happy/1010.png  
  inflating: /content/train_set/aug_happy/1011.png  
  inflating: /content/train_set/aug_happy/1012.png  
  inflating: /content/train_set/aug_happy/1013.png  
  inflating: /content/train_set/aug_happy/1014.png  
  inflating: /content/train_set/aug_happy/1015.png  
  inflating: /content/train_set/aug_happy/1016.png  
  inflating: /content/train_

In [ ]:
INPUT_SHAPE = (28, 28) 
BATCH_SIZE = 32
DATASET_PATH = "/content/train_set/"
VALIDATION_PATH = "/content/val_set/"

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_directory(
        DATASET_PATH,
        target_size=(SIZE, SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        VALIDATION_PATH,
        target_size=(SIZE, SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 3560 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'aug_happy': 0, 'aug_notr': 1}

# **Proposal Model**

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

CHECKPOINT_PATH = "/content/saved_models2/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model_without_pretrain = model.fit(
        train_generator,
        epochs=50,
        batch_size=BATCH_SIZE,
        validation_data=validation_generator,
        callbacks=callbacks_list)

model.save(CHECKPOINT_PATH+"emotion_model.h5")

Epoch 1/50
112/112 [==============================] - ETA: 0s - loss: 0.7438 - accuracy: 0.6039
Epoch 00001: val_accuracy improved from -inf to 0.53000, saving model to /content/saved_models2/weights-improvement-01-0.530.hdf5
112/112 [==============================] - 36s 301ms/step - loss: 0.7438 - accuracy: 0.6039 - val_loss: 0.6944 - val_accuracy: 0.5300
Epoch 2/50
112/112 [==============================] - ETA: 0s - loss: 0.5752 - accuracy: 0.7008
Epoch 00002: val_accuracy did not improve from 0.53000
112/112 [==============================] - 33s 298ms/step - loss: 0.5752 - accuracy: 0.7008 - val_loss: 0.8079 - val_accuracy: 0.4950
Epoch 3/50
112/112 [==============================] - ETA: 0s - loss: 0.5380 - accuracy: 0.7208
Epoch 00003: val_accuracy did not improve from 0.53000
112/112 [==============================] - 33s 296ms/step - loss: 0.5380 - accuracy: 0.7208 - val_loss: 0.8473 - val_accuracy: 0.4950
Epoch 4/50
112/112 [==============================] - ETA: 0s - loss: 

# **ResNet**

In [ ]:
INPUT_SHAPE = (48, 48) 
from tensorflow.keras.applications import resnet50
resnet = resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=IMAGE_SIZE+ [3])

for layer in resnet.layers:
    layer.trainable = False

x = Flatten()(resnet.output)
prediction = Dense(2, activation='softmax')(x)  
model_resnet = Model(inputs=resnet.input, outputs=prediction)
model_resnet.summary()
model_resnet.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

CHECKPOINT_PATH = "/content/saved_models3/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"resnet/weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
#early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
callbacks_list = [checkpoint]


# fit the model
model_with_resnet = model_resnet.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

model_resnet.save(CHECKPOINT_PATH+"resnet_balanced.h5")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


112/112 [==============================] - 35s 283ms/step - loss: 1.2361 - accuracy: 0.5253 - val_loss: 2.0789 - val_accuracy: 0.5000
Epoch 2/50
112/112 [==============================] - ETA: 0s - loss: 0.8232 - accuracy: 0.5632
Epoch 00002: val_accuracy improved from 0.50000 to 0.51750, saving model to /content/saved_models3/resnet/weights-improvement-02-0.517.hdf5
112/112 [==============================] - 29s 257ms/step - loss: 0.8232 - accuracy: 0.5632 - val_loss: 0.8788 - val_accuracy: 0.5175
Epoch 3/50
112/112 [==============================] - ETA: 0s - loss: 0.7728 - accuracy: 0.5941
Epoch 00003: val_accuracy improved from 0.51750 to 0.60750, saving model to /content/saved_models3/resnet/weights-improvement-03-0.608.hdf5
112/112 [==============================] - 29s 257ms/step - loss: 0.7728 - accuracy: 0.5941 - val_loss: 0.6501 - val_accuracy: 0.6075
Epoch 4/50
112/112 [==============================] - ETA: 0s - loss: 0.8977 - accuracy: 0.5624
Epoch 00004: val_accuracy impr

# **MobileNet**

In [ ]:
import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization   
mobileNet=keras.applications.mobilenet_v2.MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in mobileNet.layers[:1]:
    layer.trainable = False

mb=Flatten()(mobileNet.output)
mb=Dense(128,activation='relu')(mb)
mb=BatchNormalization()(mb)

mb=Dense(64,activation='relu')(mb)
mb=BatchNormalization()(mb)
prediction = Dense(2, activation='softmax')(mb)

# create a model object
model_mb = Model(inputs=mobileNet.input, outputs=prediction)

# view the structure of the model
model_mb.summary()

# tell the model what cost and optimization method to use
model_mb.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
CHECKPOINT_PATH = "/content/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"mobilenet/weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# fit the model
model_with_mb = model_mb.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

model_mb.save(CHECKPOINT_PATH+"mb_balanced.h5")

9420800/9406464 [==============================] - 0s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                             

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


112/112 [==============================] - 51s 394ms/step - loss: 0.4463 - accuracy: 0.7975 - val_loss: 3.5979 - val_accuracy: 0.6250
Epoch 2/50
112/112 [==============================] - ETA: 0s - loss: 0.1994 - accuracy: 0.9222
Epoch 00002: val_accuracy did not improve from 0.62500
112/112 [==============================] - 41s 369ms/step - loss: 0.1994 - accuracy: 0.9222 - val_loss: 5.7550 - val_accuracy: 0.5475
Epoch 3/50
112/112 [==============================] - ETA: 0s - loss: 0.1163 - accuracy: 0.9565
Epoch 00003: val_accuracy did not improve from 0.62500
112/112 [==============================] - 41s 367ms/step - loss: 0.1163 - accuracy: 0.9565 - val_loss: 5.1777 - val_accuracy: 0.5600
Epoch 4/50
112/112 [==============================] - ETA: 0s - loss: 0.0696 - accuracy: 0.9784
Epoch 00004: val_accuracy did not improve from 0.62500
112/112 [==============================] - 41s 367ms/step - loss: 0.0696 - accuracy: 0.9784 - val_loss: 5.9247 - val_accuracy: 0.5800
Epoch 5/50


In [ ]:
score = model_mb.evaluate(validation_generator)
print("Validation loss: ", score[0])
print("Validation accuracy: ",score[1])

13/13 [==============================] - 1s 92ms/step - loss: 0.3433 - accuracy: 0.9425
Validation loss:  0.3432932198047638
Validation accuracy:  0.9424999952316284


# **VGG**

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import tensorflow
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization

#preprocessing part

# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights

for layer in vgg.layers[:5]:
    layer.trainable = False

# our layers - you can add more if you want
v=Flatten()(vgg.output)
v=Dense(128,activation='relu')(v)
v=BatchNormalization()(v)
v=Dropout(0.25)(v)

v=Dense(64,activation='relu')(v)
v=BatchNormalization()(v)
prediction = Dense(2, activation='softmax')(v)

# create a model object
model_vgg = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model_vgg.summary()

# tell the model what cost and optimization method to use
model_vgg.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

CHECKPOINT_PATH="/content/vgg/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"vgg/vgg-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


# fit the model
model_with_vgg = model_vgg.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

model_vgg.save(CHECKPOINT_PATH+"vgg_balanced.h5")

58900480/58889256 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
loss_pro = [0.7438,0.5752,0.5380,0.5095,0.4634,0.4456,0.4140,0.3973,0.3619,0.3412,0.3123,]
accuracy_pro = []

score = model_vgg.evaluate(validation_generator)
print("Validation loss: ", score[0])
print("Validation accuracy: ",score[1])

13/13 [==============================] - 3s 241ms/step - loss: 6.1534 - accuracy: 0.8425
Validation loss:  6.153438568115234
Validation accuracy:  0.8424999713897705


# **EfficentNet**

In [ ]:
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization    
effNet= EfficientNetB1(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)

for layer in effNet.layers[:5]:
    layer.trainable = False

# our layers - you can add more if you want
ef=Flatten()(effNet.output)
ef=Dense(128,activation='relu')(ef)
ef=BatchNormalization()(ef)
ef=Dropout(0.25)(ef)

ef=Dense(64,activation='relu')(ef)
ef=BatchNormalization()(ef)
prediction = Dense(2, activation='softmax')(ef)

# create a model object
model_eff = Model(inputs=effNet.input, outputs=prediction)

# view the structure of the model
model_eff.summary()

# tell the model what cost and optimization method to use
model_eff.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


checkpoint = ModelCheckpoint("/content/weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# fit the model
model_with_effNet = model_eff.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

model_eff.save(CHECKPOINT_PATH+"eff_imbalanced.h5")

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                            

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


112/112 [==============================] - 121s 906ms/step - loss: 0.4925 - accuracy: 0.7677 - val_loss: 1.0826 - val_accuracy: 0.5000
Epoch 2/50
112/112 [==============================] - ETA: 0s - loss: 0.1874 - accuracy: 0.9287
Epoch 00002: saving model to /content/weights-improvement-02-0.500.hdf5
112/112 [==============================] - 96s 856ms/step - loss: 0.1874 - accuracy: 0.9287 - val_loss: 1.2845 - val_accuracy: 0.5000
Epoch 3/50
112/112 [==============================] - ETA: 0s - loss: 0.1288 - accuracy: 0.9511
Epoch 00003: saving model to /content/weights-improvement-03-0.500.hdf5
112/112 [==============================] - 96s 857ms/step - loss: 0.1288 - accuracy: 0.9511 - val_loss: 1.1255 - val_accuracy: 0.5000
Epoch 4/50
112/112 [==============================] - ETA: 0s - loss: 0.0964 - accuracy: 0.9654
Epoch 00004: saving model to /content/weights-improvement-04-0.550.hdf5
112/112 [==============================] - 96s 856ms/step - loss: 0.0964 - accuracy: 0.9654 -

In [ ]:
score = model_eff.evaluate(validation_generator)
print("Validation loss: ", score[0])
print("Validation accuracy: ",score[1])

13/13 [==============================] - 3s 228ms/step - loss: 0.0544 - accuracy: 0.9800
Validation loss:  0.054370760917663574
Validation accuracy:  0.9800000190734863
